In [130]:
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def readTrainData():
    data = []
    for line in open("data/training.json", 'r'):
        data.append(json.loads(line))
    x_train = []
    y_train = []
    for i in range(1, data[0]):
        #x_train.append( [x.lower() for x in data[i]['question'].replace('?','').split() + data[i]['excerpt'].replace('?','').split()] ) 
        x_train.append( data[i]['question'] + data[i]['excerpt'] )
        y_train.append( data[i]['topic'] )
    return x_train, y_train

def readTestData():
    num_tests  = int(input())
    x_test = []
    for i in range(num_tests):
        data = json.loads(input())
        x_test.append( data['question'] + data['excerpt'] )
    return num_tests, x_test  
        
def training(num_vocab, x_train, y_train):
    #vectorModel = TfidfVectorizer(max_features = num_vocab, ngram_range=(1,5))
    vectorModel1 = CountVectorizer(max_features = num_vocab).fit(x_train)
    x = vectorModel1.transform(x_train)
    vectorModel2 = TfidfTransformer().fit(x)
    x = vectorModel2.transform(x).toarray()
    
    log = make_pipeline( StandardScaler(with_mean=False), LogisticRegression(solver="newton-cg", random_state=42, multi_class='multinomial', penalty = 'l2', class_weight='balanced') )
    log.fit(x, y_train)
    nbM = make_pipeline( MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) )
    nbM.fit(x,y_train)
    nbG = make_pipeline( GaussianNB(var_smoothing=1e-06) )
    nbG.fit(x,y_train)
    #eclf = VotingClassifier(estimators=[('log', log), ('nbM', nbM), ('nbG', nbG)], voting='hard')
    #eclf.fit(x,y_train)
    #return vectorModel1, vectorModel2, eclf, np.mean(log.predict(x) == y_train), np.mean(nbM.predict(x) == y_train), \
    #                    np.mean(nbG.predict(x) == y_train),  np.mean(eclf.predict(x) == y_train)
    return vectorModel1, vectorModel2, log, np.mean(log.predict(x) == y_train)
                    
num_vocab = 1500
x_train, y_train = readTrainData()
vectorModel1, vectorModel2, eclf, mean_log = training(num_vocab, x_train, y_train)
print(mean_log)
#vectorModel1, vectorModel2, eclf, mean_log, mean_nbN, mean_nbG, mean_eclf = training(num_vocab, x_train, y_train)
#print(mean_log, mean_nbN, mean_nbG, mean_eclf)

num_tests, x_test = readTestData()
x = vectorModel1.transform(x_test)
x = vectorModel2.transform(x).toarray()

res = eclf.predict(x)
for type in res:
    print(type)

0.9999505391235533
1
{"question":"Frequency Inverter LS IS5","excerpt":"I have been working with a IS5 frequency inverter. I'm trying to use the Sensorless Torque control, for a Wind Turbine Emulator, via an external reference. The issue is, when I turn the inverter on ...\r\n        "}
electronics
